In [ ]:
import netCDF4 as nc
import matplotlib.pyplot as plt
import numpy as np
from math import *
import pandas as pd
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import re

In [ ]:
temp_sp = xr.open_dataset("temp_sp.nc").to_array()
speed100 = xr.open_dataset("ws100.nc")
speed100 = np.sqrt(speed100.u100**2+speed100.v100**2).values
#se = [0,251,251,480,1688,2177,2424,2896,3632,3880,5800,6048,6046,6296,7008,7256,7984,8224,9192,9432,10400,10888,11136,11360]
mtime = [250,314,1908,2678,3765,5988,6075,7214,8180,9247,10850,11227]

In [ ]:
# lat:47~64 lon:-25 7
area = []
axes = []
nlon = 0; nlat = 0
# top-left to lower-right
for lat in temp_sp.latitude.values:
    nlon = 0
    for lon in temp_sp.longitude.values:
        if  47 <= lat <= 64 and -25 <= lon <= 7:
            r = cos((lat/180)*np.pi)*6371
            c = 2*np.pi*r
            #km * km
            area.append(c/(4*360)*(2*np.pi*6371/360/4))
            axes.append((nlat,nlon))
        nlon += 1
    nlat += 1

lat_lon = []
for lat in temp_sp.latitude.values:
    for lon in temp_sp.longitude.values:  
        if  47 <= lat <= 64 and -25 <= lon <= 7:
            lat_lon.append((lat,lon))



In [ ]:
threshold_mslp = 1000.0592572373254
threshold_ke = 0.0934464621112073
threshold_per_ke = 0.6200043205024015

In [ ]:
record_time = []
overall_e925750 = []
overall_e750500 = []
overall_e500250 = []
overall_e925250 = []
overall_pe925750 = []
overall_pe750500 = []
overall_pe500250 = []
overall_pe925250 = []
overall_mslp = []
date_list = []
date_number = []

In [ ]:
# time series
# get stormy period and non-stormy period
g = 9.80665
datafilename = ["2016_Jan_Feb.nc","2016_Oct_Dec.nc","2017_Jan_Feb.nc","2017_Oct_Dec.nc",
                "2018_Jan_Feb.nc","2018_Oct_Dec.nc","2019_Jan_Feb.nc","2019_Oct_Dec.nc",
               "2020_Jan_Feb.nc","2020_Oct_Dec.nc","2021_Jan_Feb.nc","2021_Oct_Dec.nc"]

mslp_datafilename = ["2016_mslp_Jan_Feb.nc","2016_mslp_Oct_Dec.nc","2017_mslp_Jan_Feb.nc","2017_mslp_Oct_Dec.nc",
                    "2018_mslp_Jan_Feb.nc","2018_mslp_Oct_Dec.nc","2019_mslp_Jan_Feb.nc","2019_mslp_Oct_Dec.nc",
                    "2020_mslp_Jan_Feb.nc","2020_mslp_Oct_Dec.nc","2021_mslp_Jan_Feb.nc","2021_mslp_Oct_Dec.nc"]
previous = 0
for i in range(6,12):
    

    data = xr.open_dataset(datafilename[i])

    # energy of different volumes
    energy = []

    # calculate the energy per geopotential height
    energy_per_metre = []

    # wind speed
    ws = np.sqrt(data.v**2+data.u**2).values

    # height
    height = (data.z/g).values

    # geopotential 
    geo = data.z.values

    # area of each grid in the fixed rectangle
    # km^2
    # area get from calculating 100m energy

    # calculate the energy of different vertical volumes of each grid at different time
    n_time = 0
    for time in data.time.values:
        e_time = []
        e_time_per_metre = []
        for plev in range(len(data.level.values)-1):
            # hPa
            diff_plev = data.level.values[3-plev] - data.level.values[3-(plev+1)]
            e_plev = []
            e_plev_per_meter = []
            num = 0
            for ax in axes:
                # km^2 * m
                volume = area[num]*(height[n_time,3-plev-1,ax[0],ax[1]]-height[n_time,3-plev,ax[0],ax[1]])
                # hPa * s^2 * m^-2
                density = diff_plev/(geo[n_time,3-plev-1,ax[0],ax[1]]-geo[n_time,3-plev,ax[0],ax[1]])
                # km^2 * hPa * m  = km^2 * hkg * s^-2 
                e = 1/2*volume*density*(((ws[n_time,3-plev,ax[0],ax[1]]+ws[n_time,3-plev-1,ax[0],ax[1]])/2)**2)
                e_plev.append(e/10**11)
                e_plev_per_meter.append(
                    (e/(height[n_time,3-plev-1,ax[0],ax[1]]-height[n_time,3-plev,ax[0],ax[1]]))/(10**3))
                num += 1
            e_time.append(e_plev)
            e_time_per_metre.append(e_plev_per_meter)

        tot_per_metre = []
        tot = []
        num = 0
        for ax in axes:
            volume = area[num]*(height[n_time,0,ax[0],ax[1]]-height[n_time,3,ax[0],ax[1]])
            density = (925-250)/(geo[n_time,0,ax[0],ax[1]]-geo[n_time,3,ax[0],ax[1]])
            e = 1/2*volume*density*(((ws[n_time,0,ax[0],ax[1]]+ws[n_time,3,ax[0],ax[1]])/2)**2)
            tot.append(e/10**11)
            tot_per_metre.append((e/(height[n_time,0,ax[0],ax[1]]-height[n_time,3,ax[0],ax[1]]))/(10**3))
            num+=1
        e_time.append(tot)
        e_time_per_metre.append(tot_per_metre)

        energy.append(e_time)
        energy_per_metre.append(e_time_per_metre)
        n_time += 1

    # calculate total energy of storm at each time
    e_925_750 = []
    e_750_500 = []
    e_500_250 = []
    e_tot = []

    # per metre
    e_925_750_per_metre = []
    e_750_500_per_metre = []
    e_500_250_per_metre = []
    e_925_250_per_metre = []

    for time in range(len(data.time.values)):
        e_925_750.append(sum(energy[time][0]))
        overall_e925750.append(sum(energy[time][0]))
        e_925_750_per_metre.append(sum(energy_per_metre[time][0])/len(energy_per_metre[time][0]))
        overall_pe925750.append(sum(energy_per_metre[time][0])/len(energy_per_metre[time][0]))
        e_750_500.append(sum(energy[time][1]))
        overall_e750500.append(sum(energy[time][1]))
        e_750_500_per_metre.append(sum(energy_per_metre[time][1])/len(energy_per_metre[time][1]))
        overall_pe750500.append(sum(energy_per_metre[time][1])/len(energy_per_metre[time][1]))
        e_500_250.append(sum(energy[time][2]))
        overall_e500250.append(sum(energy[time][2]))
        e_500_250_per_metre.append(sum(energy_per_metre[time][2])/len(energy_per_metre[time][2]))
        overall_pe500250.append(sum(energy_per_metre[time][2])/len(energy_per_metre[time][2]))
        e_tot.append(np.sum(energy[time][0])+np.sum(energy[time][1])+np.sum(energy[time][2]))
        overall_e925250.append(np.sum(energy[time][0])+np.sum(energy[time][1])+np.sum(energy[time][2]))
        e_925_250_per_metre.append(sum(energy_per_metre[time][3])/len(energy_per_metre[time][3]))
        overall_pe925250.append(sum(energy_per_metre[time][3])/len(energy_per_metre[time][3]))

    # using kinetic energy per meter to observe the strongest stormy period
    record = []
    mslp_date = xr.open_dataset(mslp_datafilename[i])
    mslp_date = (mslp_date.msl/100).values

    average_mslp = []
    
    for time in range(len(data.time)):
        mslp = 0
        for value in axes:
            mslp += mslp_date[time,value[0],value[1]]
        mslp = mslp/len(axes)
        average_mslp.append(mslp)
        overall_mslp.append(mslp)
        if mslp < threshold_mslp and e_925_750_per_metre[time] > threshold_per_ke and e_925_750[time]>threshold_ke:
            record.append(data.time.values[time])
            date_number.append(time+previous)

    plt.figure(figsize = (13,9),dpi = 700)
    plt.plot(data.time.values, e_925_750, color = "c",label = "925hPa ~ 750hPa")
    plt.plot(data.time.values, e_750_500, color = "r",label = "750hPa ~ 500hPa")
    plt.plot(data.time.values, e_500_250, color = "g", label = "500hPa ~ 250hPa")
    plt.plot(data.time.values, e_tot, color = "black",label = "925hPa ~ 250hPa")
    for i in record:
        plt.axvline(i,linestyle = 'dashed',linewidth = 0.2)
    plt.ylabel("Power (unit : $10^19$ J)")
    plt.legend(loc = 'upper left')

    axis2 = plt.twinx()
    axis2.plot(data.time.values,average_mslp,color = "orange",label = "MSLP")
    axis2.set_ylabel('Pressure (unit : hPa)')
    plt.legend(loc = 'upper right')
    plt.title("Energy from "+str(data.time.values[0])[0:13]+" to "+str(data.time.values[-1])[0:13])

    plt.show()

    print("----------------------------------------------------------------------------------------------------")

    plt.figure(figsize = (13,9),dpi = 700)
    plt.plot(data.time.values, e_925_750_per_metre, color = "c",label = "925hPa ~ 750hPa")
    plt.plot(data.time.values, e_750_500_per_metre, color = "r",label = "750hPa ~ 500hPa")
    plt.plot(data.time.values, e_500_250_per_metre, color = "g", label = "500hPa ~ 250hPa")
    plt.plot(data.time.values, e_925_250_per_metre, color = "black",label = "925hPa ~ 250hPa")
    for i in record:
        plt.axvline(i,linestyle = 'dashed',linewidth = 0.2)
    plt.ylabel("Power (unit : $10^11$ J)")
    plt.legend(loc = 'upper left')

    axis2 = plt.twinx()
    axis2.plot(data.time.values,average_mslp,color = "orange",label = "MSLP")
    axis2.set_ylabel('Pressure (unit : hPa)')
    plt.legend(loc = 'upper right')
    plt.title("Energy per Geopotential Meter from "+str(data.time.values[0])[0:13]+" to "+str(data.time.values[-1])[0:13])

    plt.show()
    
    record_time.append(record)
    
    for time in data.time.values:
        date_list.append(str(time)[0:13])
    
    previous += len(data.time.values)

In [ ]:
plt.figure(figsize = (16,9),dpi = 800)
plt.bar(date_list[0:1208],overall_pe925750[0:1208],color = "c",label = "925hPa~750hPa energy per geopotential height",width = 0.3 )
plt.plot(date_list[0:1208],overall_e925750[0:1208],color = "orangered",label = "925hPa~750hPa energy")

plt.ylabel("Energy per geopotential height (unit : $10^{11}$ J)   /   Energy (unit : $10^{19}$ J)")
plt.legend(loc = "upper left")

xticks = list(range(0,1208,120))
plt.xticks(xticks,rotation = 280)

axis2 = plt.twinx()
axis2.plot(date_list[0:1208],overall_mslp[0:1208],color = "orange",label = "MSLP")
axis2.set_ylabel('Pressure (unit : hPa)')
plt.legend(loc = 'upper right')


for i in date_number:
    if i >= 1208:
        break
    plt.axvline(date_list[i],linestyle = 'dashed',linewidth = 0.4)
plt.title("Stormy and non-stormy periods in 2019")
plt.show()

In [ ]:
# check 1988-02-28 0:00 anti-cyclone with high centre pressure level and high wind speed
# the interaction between cyclone and anti-cyclone may have high wind speed
# single level data

datafilename = ["2016_1_2_p.nc","2016_10_12_p.nc","2017_1_2_p.nc","2017_10_12_p.nc",
                    "2018_1_2_p.nc","2018_10_12_p.nc","2019_1_2_p.nc","2019_10_12_p.nc",
                    "2020_1_2_p.nc","2020_10_12_p.nc","2021_1_2_p.nc","2021_10_12_p.nc"]

tt = 370


check = xr.open_dataset(datafilename[8])

print(check.time.values[tt])
msl = check.msl[tt,:,:]/100

ws = np.sqrt(check.u100**2+check.v100**2)


fig = plt.figure(figsize = [12,12],dpi = 550)
ax = plt.axes(projection = ccrs.Mercator())

#ax.set_extent([-75., 20., 35., 75.])
ax.set_extent([-50., 20., 45., 70.])
cp = ax.contour(msl.longitude[100:], msl.latitude[20:141], msl[20:141,100:],
                   np.arange(920, 1040, 5), cmap='RdYlBu', transform=ccrs.PlateCarree())

plt.clabel(cp,inline=1,fontsize=12,fmt='%1.0f',inline_spacing=1, colors='black')

pc = ax.contourf(check.longitude[100:], check.latitude[20:141],ws[tt,20:141,100:], np.arange(0, 40 , 5),
                 cmap='coolwarm', transform=ccrs.PlateCarree())

plt.colorbar(pc, shrink=0.7, label='wind speed (m/s)')

grid = ax.gridlines(draw_labels=True,linestyle = "--")

region = []
for i in lat_lon:
    if i[0] == 47 or i[0] == 64 or i[1]==-25 or i[1] == 7:
        region.append(i)
    
plt.scatter([value[1] for value in region],[value[0] for value in region],
           marker = 'o',s= 4,color = "green",transform = ccrs.PlateCarree())

ax.coastlines(resolution = '110m', color ="black",linewidth = 1.)
#ax.add_feature(cfeature.LAND)
plt.title(str(check.time.values[tt])[0:13])
plt.show()

In [ ]:
all_turbine = []
all_offshore = []
all_onshore = []
all_energy100m = []

all_e_925_750 = []
all_e_750_500 = []
all_e_500_250 = []
all_e_925_250 = []

all_ep_925_750 = []
all_ep_750_500 = []
all_ep_500_250 = []
all_ep_925_250 = []

In [ ]:
datafilename = ["2016_1_2_p.nc","2016_10_12_p.nc","2017_1_2_p.nc","2017_10_12_p.nc",
                    "2018_1_2_p.nc","2018_10_12_p.nc","2019_1_2_p.nc","2019_10_12_p.nc",
                    "2020_1_2_p.nc","2020_10_12_p.nc","2021_1_2_p.nc","2021_10_12_p.nc"]

e_datafilename = ["2016_Jan_Feb.nc","2016_Oct_Dec.nc","2017_Jan_Feb.nc","2017_Oct_Dec.nc",
                "2018_Jan_Feb.nc","2018_Oct_Dec.nc","2019_Jan_Feb.nc","2019_Oct_Dec.nc",
               "2020_Jan_Feb.nc","2020_Oct_Dec.nc","2021_Jan_Feb.nc","2021_Oct_Dec.nc"]

g = 9.80665

for i in range(6,12):
    
    data = xr.open_dataset(datafilename[i])
    mslp = data.msl.values/100
    
    temp2m = data.t2m.values
    
    sp = data.sp.values/100
    
    # decrease at 0.65K/100m
    temp100m = temp2m - 0.65*98/100
    tempsuf = temp2m + 0.65*2/100
    
    # p(100) = p(surface) * (1-0.0065*delta/T(2m))^5.225
    # p(surface) hPa
    p100m = sp*((1-0.65/tempsuf)**5.225)

    # kg/m^3
    G = 287.058
    # hPa/(J * Kg-1) = 100 J/m^3 /(J*kg-1) = kg/m^3
    density100m = (p100m/(G*temp100m)*100)
    
    ws100 = np.sqrt(data.v100**2+data.u100**2).values
    
    # from top-right cornor to lower-left cornor
    # calculate energy
    energy100m = []

    onshore = []
    offshore = []
    turbine = []
    
    # average kinetic energy in the region
    e100m = []


    # energy100m unit J
    for t in range(len(data.time.values)):
        num = 0
        Sum = 0 
        for index in axes:
            # J
            e = 1/2*density100m[t,index[0],index[1]]*area[num]*((ws100[t,index[0],index[1]])**2)*(10**6)
            Sum += e
            energy100m.append(e)
            num+=1
        e100m.append(Sum/(num))
        all_energy100m.append(Sum)
    # calculate the power from turbine
    
    tpower = []
    for t in range(len(data.time.values)):
        num = 0
        Sum = 0
        ons = 0
        ofs = 0
        ons_num = 0
        ofs_num = 0
        for index in axes:
            # lower in cut-in or higher than cut-out
            if 50 <= lat_lon[num][0] <=57 and -13 <= lat_lon[num][1] <=-5:
                ons_num += 1
            else:
                ofs_num += 1
            
            if ws100[t,index[0],index[1]] > max(curve.keys()) or ws100[t,index[0],index[1]] < min(curve.keys()):
                tpower.append(0)
                num += 1
                continue
            # kWh
            
            speed = 0
            if ws100[t,index[0],index[1]] < int(ws100[t,index[0],index[1]])+0.25:
                speed = int(ws100[t,index[0],index[1]]) 
            elif ws100[t,index[0],index[1]] > int(ws100[t,index[0],index[1]])+0.75:
                speed = ceil(ws100[t,index[0],index[1]])
            else :
                speed = int(ws100[t,index[0],index[1]])+0.5
            e = curve[speed]
            # J kw -> kJ/s default 1 second
            e = e * (area[num] / 0.01) * (6 /37) * (10**3)
            tpower.append(e)
            Sum += e
            
            
            if 50 <= lat_lon[num][0] <=57 and -13 <= lat_lon[num][1] <=-5:
                ons += e
            else:
                ofs += e
            num += 1
            
        turbine.append(Sum/num)
        onshore.append(ons/ons_num)
        offshore.append(ofs/ofs_num)
        
        all_turbine.append(Sum)
        all_offshore.append(ofs)
        all_onshore.append(ons)
    
    

    e_data = xr.open_dataset(e_datafilename[i])

    # energy of different volumes
    energy = []

    # calculate the energy per geopotential height
    energy_per_metre = []

    # wind speed
    ws = np.sqrt(e_data.v**2+e_data.u**2).values

    # height
    height = (e_data.z/g).values

    # geopotential 
    geo = e_data.z.values

    # area of each grid in the fixed rectangle
    # km^2
    # area get from calculating 100m energy

    # calculate the energy of different vertical volumes of each grid at different time
    n_time = 0
    for t in e_data.time.values:
        e_time = []
        e_time_per_metre = []
        for plev in range(len(e_data.level.values)-1):
            # hPa
            diff_plev = e_data.level.values[3-plev] - e_data.level.values[3-(plev+1)]
            e_plev = []
            e_plev_per_meter = []
            num = 0
            for ax in axes:
                # km^2 * m
                volume = area[num]*(height[n_time,3-plev-1,ax[0],ax[1]]-height[n_time,3-plev,ax[0],ax[1]])
                # hPa * s^2 * m^-2
                density = diff_plev/(geo[n_time,3-plev-1,ax[0],ax[1]]-geo[n_time,3-plev,ax[0],ax[1]])
                # km^2 * hPa * m  = km^2 * hkg * s^-2 
                e = 1/2*volume*density*(((ws[n_time,3-plev,ax[0],ax[1]]+ws[n_time,3-plev-1,ax[0],ax[1]])/2)**2)
                e_plev.append(e)
                e_plev_per_meter.append(
                    e/(height[n_time,3-plev-1,ax[0],ax[1]]-height[n_time,3-plev,ax[0],ax[1]]))
                num += 1
            e_time.append(e_plev)
            e_time_per_metre.append(e_plev_per_meter)

        tot_per_metre = []
        tot = []
        num = 0
        for ax in axes:
            volume = area[num]*(height[n_time,0,ax[0],ax[1]]-height[n_time,3,ax[0],ax[1]])
            density = (925-250)/(geo[n_time,0,ax[0],ax[1]]-geo[n_time,3,ax[0],ax[1]])
            e = 1/2*volume*density*(((ws[n_time,0,ax[0],ax[1]]+ws[n_time,3,ax[0],ax[1]])/2)**2)
            tot.append(e)
            tot_per_metre.append(e/(height[n_time,0,ax[0],ax[1]]-height[n_time,3,ax[0],ax[1]]))
            num += 1
        e_time.append(tot)
        e_time_per_metre.append(tot_per_metre)

        energy.append(e_time)
        energy_per_metre.append(e_time_per_metre)
        n_time += 1

    # calculate total energy of storm at each time
    e_925_750 = []
    e_750_500 = []
    e_500_250 = []
    e_tot = []

    # per metre
    e_925_750_per_metre = []
    e_750_500_per_metre = []
    e_500_250_per_metre = []
    e_925_250_per_metre = []

    for t in range(len(e_data.time.values)):
        e_925_750.append(sum(energy[t][0]))
        all_e_925_750.append(sum(energy[t][0]))
        e_925_750_per_metre.append(sum(energy_per_metre[t][0])/len(energy_per_metre[t][0]))
        all_ep_925_750.append(sum(energy_per_metre[t][0])/len(energy_per_metre[t][0]))
        
        e_750_500.append(sum(energy[t][1]))
        all_e_750_500.append(sum(energy[t][1]))
        e_750_500_per_metre.append(sum(energy_per_metre[t][1])/len(energy_per_metre[t][1]))
        all_ep_750_500.append(sum(energy_per_metre[t][1])/len(energy_per_metre[t][1]))
        
        e_500_250.append(sum(energy[t][2]))
        all_e_500_250.append(sum(energy[t][2]))
        e_500_250_per_metre.append(sum(energy_per_metre[t][2])/len(energy_per_metre[t][2]))
        all_ep_500_250.append(sum(energy_per_metre[t][2])/len(energy_per_metre[t][2]))
        
        e_tot.append(np.sum(energy[t][0])+np.sum(energy[t][1])+np.sum(energy[t][2]))
        all_e_925_250.append(np.sum(energy[t][0])+np.sum(energy[t][1])+np.sum(energy[t][2]))
        e_925_250_per_metre.append(sum(energy_per_metre[t][3])/len(energy_per_metre[t][3]))
        all_ep_925_250.append(sum(energy_per_metre[t][3])/len(energy_per_metre[t][3]))

        
    
    print(i)

In [ ]:
stormturbine = []
non_stormturbine = []
stormenergy100 = []
non_stormenergy100 = []
stormtotal = []
non_stormtotal = []

for i in range(len(date_list2)):
    if i in date_number2:
        stormturbine.append(all_turbine[i]/(10**14))
        stormenergy100.append(all_energy100m[i]/10**15)
        stormtotal.append(all_e_925_250[i]/10**11)
    
    else:
        non_stormturbine.append(all_turbine[i]/(10**14))
        non_stormenergy100.append(all_energy100m[i]/10**15)
        non_stormtotal.append(all_e_925_250[i]/(10**11))
        
plt.figure(figsize = (10,10),dpi = 500)
plt.ylabel("Turbine output power (unit:$10^{14}$ J)")
plt.boxplot([stormturbine,non_stormturbine],labels = ["Storm periods","Non-storm periods"],
            patch_artist=True,boxprops = {'facecolor':'pink'},
            notch=True,showmeans=True,meanprops = {'markerfacecolor':'orangered'},showfliers= False)
plt.show()

plt.figure(figsize = (10,10),dpi = 500)
plt.ylabel("100m energy (unit:$10^{15}$ J)")
plt.boxplot([stormenergy100,non_stormenergy100],labels = ["Storm periods","Non-storm periods"],
            patch_artist=True,boxprops = {'facecolor':'cornflowerblue'},
            notch=True,showmeans=True,showfliers= False,meanprops = {'markerfacecolor':'orangered'})
plt.show()

plt.figure(figsize = (10,10),dpi = 500)
plt.ylabel("Total energy in the atmosphere (unit:$10^{19}$ J)")
plt.boxplot([stormtotal,non_stormtotal],labels = ["Storm periods","Non-storm periods"],
            patch_artist=True,boxprops = {'facecolor':'paleturquoise'},
            notch=True,showmeans=True,showfliers= False,meanprops = {'markerfacecolor':'orangered'})
plt.show()

In [ ]:
stormturbine = []
non_stormturbine = []
stormenergy100 = []
non_stormenergy100 = []
stormtotal = []
non_stormtotal = []
stormoffshore = []
non_stormoffshore = []
stormonshore = []
non_stormonshore = []

for i in range(len(date_list2)):
    if i in date_number2:
        stormturbine.append(all_turbine[i]/(10**14))
        stormenergy100.append(all_energy100m[i]/(10**15))
        stormtotal.append(all_e_925_250[i]/10**11)
        stormonshore.append(all_onshore[i]/(10**14))
        stormoffshore.append(all_offshore[i]/(10**14))
    
    else:
        non_stormturbine.append(all_turbine[i]/(10**14))
        non_stormenergy100.append(all_energy100m[i]/(10**15))
        non_stormtotal.append(all_e_925_250[i]/(10**11))
        non_stormonshore.append(all_onshore[i]/(10**14))
        non_stormoffshore.append(all_offshore[i]/(10**14))

In [ ]:
print(np.sum(stormturbine)/(len(stormturbine)),np.sum(stormonshore)/(len(stormonshore)),
      np.sum(stormoffshore)/len(stormoffshore),np.sum(stormenergy100)/len(stormenergy100),
     np.sum(stormtotal)/len(stormtotal))

print(np.sum(stormturbine)/(len(stormturbine))*10**14/(np.sum(stormenergy100)/len(stormenergy100)*10**15))
print(np.sum(stormturbine)/(len(stormturbine))*10**14/(np.sum(stormtotal)/len(stormtotal)*10**19)*100)

In [ ]:
print(np.sum(non_stormturbine)/(len(non_stormturbine)),np.sum(non_stormonshore)/(len(non_stormonshore)),
      np.sum(non_stormoffshore)/len(non_stormoffshore),np.sum(non_stormenergy100)/len(non_stormenergy100),
     np.sum(non_stormtotal)/len(non_stormtotal))

print(np.sum(non_stormturbine)/(len(non_stormturbine))*10**14/(np.sum(non_stormenergy100)/len(non_stormenergy100)*10**15))
print(np.sum(non_stormturbine)/(len(non_stormturbine))*10**14/(np.sum(non_stormtotal)/len(non_stormtotal)*10**19)*100)